In [14]:
import numpy as np

from sklearn.linear_model import LinearRegression

In [15]:
rs = np.random.RandomState(0)

In [90]:
# Toy SCM
# Observational distribution
eta1 = rs.normal(size=10000)
eta2 = rs.normal(size=10000)
eta3 = rs.normal(size=10000)
eta4 = rs.normal(size=10000)
eta5 = rs.normal(size=10000)

eta6 = rs.normal(size=10000)

b = eta1
theta1 = eta2
g = 0.5 * b + eta3
r = 0.3 * g + 0.7 * theta1 + eta4
theta2 = 0.2 * g + 0.6 * b + eta5

X_disent_obs = np.stack((r, g, b, theta1, theta2), axis=1)

y_obs = r + g + b

In [91]:
# Interventional distribution
# Resample noise
eta1 = rs.normal(size=5000)
eta2 = rs.normal(size=5000)
eta3 = rs.normal(size=5000)
eta4 = rs.normal(size=5000)
eta5 = rs.normal(size=5000)

eta6 = rs.normal(size=5000)

b = eta1
theta1 = 10 * np.ones(5000)
g = 0.5 * b + eta3
r = 0.3 * g + 0.7 * theta1 + eta4
theta2 = 0.2 * g + 0.6 * b + eta5

X_disent_iv = np.stack((r, g, b, theta1, theta2), axis=1)

y_iv = r + g + b


In [92]:
regr_disent = LinearRegression()
regr_disent.fit(X_disent_obs[:9000, :], y_obs[:9000])

LinearRegression()

In [93]:
y_hat_disent_id = regr_disent.predict(X_disent_obs[9000:, :])
acc_disent_id = np.mean((y_obs[9000:] - y_hat_disent_id) ** 2)
print(f'ID error disentangled: {acc_disent_id}')

y_hat_disent = regr_disent.predict(X_disent_iv)
acc_disent = np.mean((y_iv - y_hat_disent) ** 2)
print(f'OOD error disentangled: {acc_disent}')

ID error disentangled: 5.223982903060669e-30
OOD error disentangled: 1.660367592736979e-28


In [94]:
lin_mixing = rs.uniform(size=(5, 5))

In [98]:
# Linear case
# X_ent_obs = X_disent_obs @ lin_mixing
# X_ent_iv = X_disent_iv @ lin_mixing
# Nonlinear case
X_ent_obs = (X_disent_obs @ lin_mixing) ** 3
X_ent_iv = (X_disent_iv @ lin_mixing) ** 3

In [99]:
regr_ent = LinearRegression()
regr_ent.fit(X_ent_obs[:9000,:], y_obs[:9000])

LinearRegression()

In [100]:
y_hat_ent_id = regr_ent.predict(X_ent_obs[9000:, :])
acc_ent_id = np.mean((y_obs[9000:] - y_hat_ent_id) ** 2)
print(f'ID error entangled: {acc_ent_id}')

y_hat_ent = regr_ent.predict(X_ent_iv)
acc_ent = np.mean((y_iv - y_hat_ent) ** 2)
print(f'OOD error entangled: {acc_ent}')

ID error entangled: 2.855155743448587
OOD error entangled: 1128.9651439292


In [101]:
y_hat = regr_ent.predict(X_ent_obs)
acc_id_ent = np.mean((y_obs - y_hat) ** 2)

In [102]:
acc_id_ent

2.332091030846714

In [ ]:
# Nonlinear mixing case: entangled representation results in (potentially) unbounded OOD error, while disentangled setting is robust to interventions!